In [1]:
import pandas as pd
import smtplib
import datetime
from email.mime.text import MIMEText
import logging
import logging.handlers
from dateutil.relativedelta import relativedelta
import csv
import numpy as np
from datetime import datetime

In [2]:
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s : %(levelname)s - %(message)s',
    handlers=[
        logging.handlers.RotatingFileHandler("C:/Users/00220401626/Desktop/02_theme_master/02_theme_master/03_mergecsv/merge_csv.log",
                                             maxBytes=1*1024*1024,
                                             backupCount=5)
    ]
)
logger = logging.getLogger(__name__)
logger.info('Process Start.')

In [3]:
ub_themefile = 'C:/Users/00220401626/Desktop/02_theme_master/02_theme_master/01_export/ubexport_thememaster.csv'
concat_file = 'C:/Users/00220401626/Desktop/02_theme_master/02_theme_master/02_editcsv/concat.csv'

dict_ub_theme = pd.read_csv(ub_themefile, index_col=0, encoding='cp932').to_dict(orient='index')
concat_list = pd.read_csv(concat_file, encoding='cp932').values.tolist()

In [4]:
for cl in concat_list:
    themecd = cl[0]
    if themecd in dict_ub_theme.keys():
        dict_val = dict_ub_theme[themecd]
        if pd.isnull(dict_val['最終決裁完了日時']) or datetime.strptime(dict_val['最終決裁完了日時'],'%Y/%m/%d %H:%M:%S') < datetime.strptime(cl[19],'%Y/%m/%d %H:%M:%S'):
            
            logger.debug('Update. themecd:' + themecd + ', 最終決裁完了日時: ' + str(dict_val['最終決裁完了日時']) + ', 決裁完了日: ' + str(cl[19]))
    
            if pd.isnull(dict_val['最終決裁完了日時']) or datetime.strptime(str(dict_val['最終決裁完了日時']), '%Y/%m/%d  %H:%M:%S' ) >= datetime(2023, 4, 1, 0, 0, 0):

                dict_val.update({
                    'テーマ名': cl[1],
                    '前テーマコード': cl[3],
                    'MP登録用仮テーマCD': cl[4],
                    'テーマリーダー': cl[5],
                    'テーマリーダー社員No': cl[5],
                    'テーマリーダー部署コード': cl[6],
                    'テーマリーダー部署名': cl[7],
                    'テーマ区分': cl[8],
                    'プロダクトライン': cl[9],
                    '依頼元 事業部': cl[10],
                    '4象限区分': cl[11],
                    '開発活動ステータス': cl[12],
                    '開始日': cl[13],
                    '完了予定日': cl[14],
                    '完了/中止/中断日': cl[15],
                    '抹消日': cl[16],
                    '税前貢献 開始年月': cl[17],
                    '税前貢献 終了年月': cl[18],
                    '最終決裁完了日時': cl[19],
                })
                dict_ub_theme[themecd] = dict_val        

            else: 
                dict_val.update({  
                        '最終決裁完了日時': cl[19],
                    })
                dict_ub_theme[themecd] = dict_val
        else:
            logger.debug('Skip.   themecd:' + themecd + ', 最終決裁完了日時: ' + str(dict_val['最終決裁完了日時']) + ', 決裁完了日: ' + str(cl[19]))
    else:
        logger.debug('New.    themecd:' + themecd + ', 決裁完了日: ' + str(cl[19]))
        
        dict_val = {
                '開発コード': cl[2],                                                         
                'テーマ名': cl[1],
                '前テーマコード': cl[3],
                'MP登録用仮テーマCD': cl[4],
                'テーマリーダー': cl[5],
                'テーマリーダー社員No': cl[5],
                'テーマリーダー部署コード': cl[6],
                'テーマリーダー部署名': cl[7],
                'テーマ区分': cl[8],
                'プロダクトライン': cl[9],
                '依頼元 事業部': cl[10],
                '4象限区分': cl[11],
                '開発活動ステータス':cl[12],
                '開始日' : cl[13],
                '完了予定日' : cl[14],
                '完了/中止/中断日': cl[15],
                '抹消日' : cl[16],
                '税前貢献 開始年月': cl[17],
                '税前貢献 終了年月' : cl[18],
                '最終決裁完了日時' : cl[19]
            }
        
        dict_ub_theme[themecd]=dict_val    

In [5]:
df1=pd.DataFrame(dict_ub_theme.keys(),columns=['テーマコード'])
df2=pd.json_normalize(dict_ub_theme.values())
# indexをキーに結合
df=df1.join(df2)
print(df)
# CSV出力
csv_path = 'C:/Users/00220401626/Desktop/theme_master.csv'
df.to_csv(csv_path, index=False, encoding='cp932')
logger.info('Process End.')

      テーマコード  開発コード                      テーマ名   テーマ区分         開始日       完了予定日  \
0    00052D1  00052                      共通部材  区分5_共通  2008/06/01         NaN   
1    00105D1  00105                     ＡＯ７０Ｐ  区分3_独自  2016/09/30         NaN   
2    00105R1  00105               (残額回収)ＡＯ７０Ｐ  区分3_独自  2016/09/30         NaN   
3    00203D1  00203                   生接合技術開発  区分1_受託  2010/03/23  2011/03/30   
4    00212D1  00212                   外周研削盤開発  区分1_受託  2009/04/20  2011/03/30   
..       ...    ...                       ...     ...         ...         ...   
803  A0257H2  A0257      SI脱脂工程における新抽出溶媒仕様の確立  区分1_受託  2023/04/06  2024/03/31   
804  A0177H3  A0177   次世代LTCCパッケージ向け微細配線工法の開発  区分1_受託  2023/04/17  2024/03/31   
805  A0248H2  A0248  CMOSﾊﾟｯｹｰｼﾞ事業部倍増めっきライン構築  区分1_受託  2023/04/24  2024/03/31   
806  A0243H2  A0243  広視野微小凹凸検査　量産仕様確立に向けた設備開発  区分1_受託  2023/04/01  2024/03/31   
807  A0286D2  A0286        プログラミング不要な検査ソフトの開発  区分1_受託  2023/04/13  2024/09/30   

             依頼元 事業部 税前貢献 開